# **논문 요약기 V1.0.0**
> Written by _Team 두부_

In [ ]:
#@title ## **1. 라이브러리 설치하기**
#@markdown 왼쪽의 __"실행"__ 버튼을 눌러 필요한 라이브러리들을 설치해주세요. <br/> 설치에는 다소 시간이 소요될 수 있어요. <br/>
#@markdown > **설치 라이브러리 모음**
#@markdown > <br/> - `tqdm` : Progress bar for loops and tasks.
#@markdown > <br/> - `pyfiglet` : `ASCII` art text generator.
#@markdown > <br/> - `pexpect` : Library for controlling and automating other programs.
#@markdown > <br/> - `openai` : To use `ChatGPT` API 
#@markdown > <br/> - `tabulate` : Pretty-print tabular data.
#@markdown > <br/> - `fitz` : Python binding for `MuPDF`.
#@markdown > <br/> - `pdf2image` : A python module that wraps `pdftoppm` and `pdftocairo` to convert PDF to a PIL Image object.
#@markdown > <br/> - `pytesseract` : Wrapper for Google's `Tesseract-OCR` Engine.
#@markdown > <br/> - `frontend` : Visualizations for Jupyter notebooks.
#@markdown > <br/> - `opencv-contrib-python` : OpenCV's extended modules for computer vision.
#@markdown > <br/> - `pillow` : Python Imaging Library (Fork) for opening, manipulating, and saving many different image file formats.
#@markdown > <br/> - `chatgpt-wrapper` : An useful open-source unofficial Power CLI, Python API and Flask API that lets us interact programmatically with ChatGPT/GPT4.

!pip install tqdm -qq
from tqdm.notebook import tqdm_notebook

packages = ['pyfiglet', 'pexpect', 'openai', 'tabulate', 'fitz', 'pdf2image', 'pytesseract', 'frontend', 'opencv-contrib-python', 'pillow']
for package in tqdm_notebook(packages, desc="Installing packages"):
    !pip install $package -qq &> /dev/null

!pip install git+https://github.com/mmabrouk/chatgpt-wrapper -qq &> /dev/null # Install `chatgpt-wrapper`
!apt-get install poppler-utils -qq &> /dev/null # To progress PDF OCR procedures in Colab
!sudo apt-get install tesseract-ocr -qq &> /dev/null # Configure `tesseract-ocr` in Colab
import os
os.environ['PATH'] += ':/usr/local/bin'
!echo "INFO: 설치가 완료되었어요!"


In [ ]:
#@title ## **2. API Key 입력하기** 
#@markdown https://platform.openai.com/account/api-keys 에 접속하여, API key 값을 복사해주세요. <br/> __"실행"__ 버튼을 클릭한 후, 복사한 API key 값을 입력해주세요. 

import os 
import openai 

your_api = input("INFO: 복사한 API key를 붙여넣어주세요: ")
os.environ["OPENAI_API_KEY"] = your_api
os.system('cls' if os.name == 'nt' else 'clear')
print("INFO: API key가 성공적으로 등록되었어요!")

In [ ]:
#@title ##**3. ChatGPT API 실행 준비하기** 
#@markdown __"실행"__ 버튼을 입력하여 ChatGPT API를 실행할 준비를 완료하세요.

!apt-get install sqlite3 &> /dev/null

import pexpect
import os
from tqdm.auto import tqdm

# Set the path to the SQLite binary
sqlite_path = "/lib/x86_64-linux-gnu/libsqlite3.so.0"

# Set the path to the SQLite database file
db_path = "////root/.local/share/chatgpt-wrapper/profiles/default/storage.db"

# Define the SQLite command to execute
sqlite_command = "{} {}"

# Start the SQLite process and connect to the database
process = pexpect.spawn("chatgpt", encoding="utf-8")
process.expect("Enter username")

# Send the username to the SQLite process and expect a prompt
process.sendline("UserID1")
process.expect("Enter email")

# Send a blank password to the SQLite process and expect a prompt
process.sendline("")
process.expect("Enter password")

# Send a blank email to the SQLite process and expect a prompt
process.sendline("")
process.expect("Provide a prompt")
print(process.before)

# Send a command to log in using pexpect
process.sendline("/login UserID1")
process.expect("userid1@")

# Send a command to exit the SQLite process
process.sendline("/exit")
process.wait()

tqdm.write("INFO: ChatGPT API를 사용할 준비가 완료되었어요.")

In [ ]:
#@title ## **4. 인풋 파일 Colab에 업로드하기**
#@markdown __"실행"__ 버튼을 눌러 논문 요약을 진행 할 파일을 업로드해주세요. <br/>
#@markdown 인풋 파일로는 `.md`, `.txt`, 그리고 `.pdf` 세 가지 형태의 파일 타입을 지원하고 있습니다. <br/>
#@markdown > __⚠️ NOTE__: PDF 파일을 선택하면, OCR 스캐닝을 이용해 이미지로부터 텍스트들을 불러오게 됩니다. 따라서 직접 정리하여 놓은 텍스트 파일보다 정확도가 다소 떨어질 수 있다는 점, 미리 알려드립니다.

from google.colab import files
uploaded = files.upload()

print("INFO: 파일(들)이 성공적으로 업로드되었어요!")

In [ ]:
#@title ## **5. 논문 요약기 실행하기**
#@markdown __"실행"__ 버튼을 눌러 논문 요약기를 실행해보세요.

# ------------------ Import libraries ------------------ #
import os
import sys
import glob
import pyfiglet
import numpy as np
from tabulate import tabulate
from chatgpt_wrapper import OpenAIAPI
from google.colab import files

# Manipulating PDF
import cv2
import pytesseract as tess
from pdf2image import convert_from_path
from PIL import Image
from tqdm import tqdm

# ------------------ Main code starts ------------------ #
# Print the title 
os.system('cls' if os.name == 'nt' else 'clear')
title = pyfiglet.figlet_format('PaperSumGPT', font = 'small')
print('\n')
print(title+'\n')
print('\n')
print('------------------------------------------------')
print('궁금한 점이 있으시면 제 이메일로 메일을 보내주세요.')
print('\n에러나 업데이트가 필요한 부분이 있으면 알려주세요.')
print('\n 📨 woo_go@yahoo.com')
print('\n제 깃허브 (https://github.com/wjgoarxiv/papersumgpt)를 방문하셔서 더 많은 정보를 얻어보세요.')
print('------------------------------------------------')

def main():
    # Ask user if the brought input file is a markdown file or plain text file 
    print('\n')
    file_type = int(input("""INFO: 인풋 파일로 넣고자 하는 파일의 확장자를 번호 (1~3)로 선택해주세요.

    1. Markdown (`.md`) file
    2. Plain text (`.txt`) file
    3. PDF (`.pdf`) file

    : """))
    print('\n')
    print('------------------------------------------------')

    # Print the list of files in the current directory
    if file_type == 1:
        file_list = glob.glob('./*.md')
        file_list = [file.split('\\')[-1] for file in file_list]
        file_list.sort()

    elif file_type == 2:
        file_list = glob.glob('./*.txt')
        file_list = [file.split('\\')[-1] for file in file_list]
        file_list.sort()

    elif file_type == 3:
        file_list = glob.glob('./*.pdf')
        file_list = [file.split('\\')[-1] for file in file_list]
        file_list.sort()

    # File not found error handling
    try: 
        if len(file_list) == 0:
            raise FileNotFoundError
        else:
            pass

    # Alert the user 
    except: 
        print('ERROR: 현재 디렉터리에 파일이 없습니다. 디렉토리를 확인해보세요.')
        print('------------------------------------------------')
        sys.exit()

    # Print the list of files in the current directory
    file_num = [] 
    for i in range(len(file_list)):
        file_num.append(i+1)
    print(tabulate({'File number': file_num, 'File name': file_list}, headers='keys', tablefmt='psql'))
    print('------------------------------------------------')
    user_input = int(input('\nINFO: 파일 번호를 선택하거나 "0"을 눌러 종료하세요: '))

    if user_input == 0:
        print("INFO: 프로그램을 종료합니다.")
        sys.exit()
    else:
        try: 
            print("INFO: 사용할 파일 이름은 다음과 같습니다: ", file_list[user_input-1])
        except:
            print("ERROR: 입력 번호가 범위를 벗어났습니다. 파일 번호를 확인하세요.")
            print("ERROR: 프로그램을 종료합니다.")
            sys.exit()

    # Ask user to turn on `verbose` mode. 
    # If the user turns on `verbose` mode, the program will print the intermediate results.
    print('------------------------------------------------')
    verbose = input("INFO: verbose mode (상세 모드)를 켜시겠어요? 상세 모드를 켜면 중간 결과를 확인해볼 수 있어요. (y/n): ")
    if verbose == 'y' or verbose == 'Y':
        verbose = True
    elif verbose == 'n' or verbose == 'N':
        verbose = False
    else:
        print("ERROR: 잘못된 응답을 입력하셨어요. 프로그램을 종료합니다...")
        sys.exit()
    print('------------------------------------------------')

    # Ask user their desired ChatGPT model
    chatgpt_model = input("""INFO: 사용할 ChatGPT 모델의 번호를 입력해주세요:

    1. default (Turbo version for ChatGPT Plus users and default version for free users)
    2. gpt4 (Only available for ChatGPT Plus users; a little bit slower than the default model)
    3. legacy (Only available for ChatGPT Plus users; an older version of the default model)

    """)
                        
    if chatgpt_model == '1':
        chatgpt_model = 'default'
    elif chatgpt_model == '2':
        chatgpt_model = 'gpt4'
    elif chatgpt_model == '3':
        chatgpt_model = 'legacy'
    else:
        print("ERROR: 입력 번호가 범위를 벗어났어요. 파일 번호를 확인해 주세요.")
        print("ERROR: 프로그램을 종료합니다. ")
        sys.exit()
    print('------------------------------------------------')

    # ------------------ Convert pdf to markdown ------------------ #
    # This process requires following processes: 
    # 1. Convert pdf to images
    # 2. Perform OCR
    # 3. Convert the images to a markdown file 

    def pdf_to_images(pdf_file):
        return convert_from_path(pdf_file, 500)
    
    def process_image(image):
        # Convert PIL image to OpenCV image
        original_image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        gray_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)
        _, threshold_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        rectangular_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (11, 11))
        dilated_image = cv2.dilate(threshold_image, rectangular_kernel, iterations=1)

        contours, hierarchy = cv2.findContours(dilated_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        copied_image = original_image.copy()

        ocr_text = ""

        for cnt in contours:
            x, y, w, h = cv2.boundingRect(cnt)
            cropped = copied_image[y:y + h, x:x + w]
            text = tess.image_to_string(cropped, config='--oem 3 --psm 1')
            ocr_text += text

        return ocr_text
    
    def perform_ocr(images):
        ocr_text = ""
        for i, image in tqdm(enumerate(images), total=len(images)):
            text = process_image(image)
            ocr_text += f"Page {i+1}:\n{text}\n\n"
        return ocr_text
    
    if file_type == 3:
        print('INFO: PDF 파일을 마크다운 파일로 변환하는 중...')
        pdf_text = perform_ocr(pdf_to_images(file_list[user_input-1]))

        with open(file_list[user_input-1] + '_markdowned.md', 'w', encoding='utf-8') as f:
            for line in pdf_text:
                f.write(line)

        file_list[user_input-1] = file_list[user_input-1] + '_markdowned.md'

        print('INFO: PDF 파일이 마크다운 파일로 변환되었습니다.')
        print('------------------------------------------------')

    # ------------------ Function starts ------------------ #

    # Initialize ChatGPT
    bot = OpenAIAPI()

    # read input file
    with open(file_list[user_input-1], 'r') as f:
        input_text = f.read()

    # Ask user maximum length of input text (if don't know, just input 3000)
    max_length = 7000

    # truncate input text into smaller parts
    input_parts = [input_text[i:i+max_length] for i in range(0, len(input_text), max_length)]

    # define initial prompt message
    initial_prompt = "Please, act as 'High-quality content abbreviator'. Since you have the input limits (OpenAI limited your input limit), you have to firstly take the all the inputs iteratively. To do this, I've already truncated the long inputs into each subpart. You'll now have to take the inputs iteratively. The important thing is that you should NOT answer directly or respond to the previous message. Make sure that you have to accomplish the task when all the inputs are given. I'll let you know if all the inputs are given."

    # send initial prompt message to ChatGPT
    print("INFO: Tossing initial prompt...")
    success, response, message = bot.ask_stream(initial_prompt) # ask_stream is required; since we have to make ChatGPT API understands the context.
    for chunk in bot.ask_stream(initial_prompt):
        print(chunk)

    # initialize response list
    response_parts = []

    # define prompt message while iterating over input parts and send them to ChatGPT
    for i, part in enumerate(input_parts):
        if i == len(input_parts) - 1:
            prompt = f"This is the {i+1}th part of the truncated input contents. And PLEASE! Do NOT answer and if you understood the input, just keep asking me to input the leftover contents.\n\n```\n{part}\n```\nThank you for providing all the inputs."
        else:
            prompt = f"This is the {i+1}th part of the truncated input contents. And PLEASE! Do NOT answer and if you understood the input, just keep asking me to input the leftover contents.\n\n```\n{part}\n```"

        print(f"INFO: 진행 중... ({i+1}/{len(input_parts)})")

    # send prompt message and prompt part to ChatGPT
        for chunk in bot.ask_stream(prompt):
            response_parts.append(chunk)

    # Handling the `verbose` mode 
        if verbose == True:
            print(f"INFO: {i+1}/{len(input_parts)} 부분의 전달된 내용: {prompt}")
            print(f"INFO: ChatGPT의 응답: {response}")
        else:
            pass


    # define final prompt message
    final_prompt = """Now, all the inputs are given to you. You should co   mbine and abbreviate all the inputs by fitting into the following markdown format. The markdown format is as follows:
    
    ------ TEMPLATE STARTS ------

    # **[TITLE]**
    (Bring the title from the foremost heading in the document. The powerful hint is that the title comes before the people who wrote the document.)

    ## **Introduction**

    ## **Methodology**
    ### **Apparatus**
    ### **Experimental procedure**
    ### **Computational procedure (if exists)**
    ### **Data analysis**

    ## **Results & discussion**

    ## **Conclusions**

    ## **Significance of this study**

    ## **Things to look out for in follow-up research**

    ### **Useful references to consider**
    ...

    ------ TEMPLATE ENDS ------

    And please, write the outputs thinking you are writing PPT slides. But NOT too simple. You have to write the outputs in a way that the readers can understand the contents easily.
    Consecutively, if possible, please find some useful references (including title and authors) from Text or Markdown input file, and re-write them into `### Useful references to consider` subheader. 
    """

    # send final prompt message to ChatGPT
    print("INFO: 마지막 프롬프트를 전달합니다...")

    # join response parts to form final response
    # If final response doesn't exist, use the last response part
    try: 
        final_response = response_parts[-1]
    except:
        print("ERROR: 목록 인덱스가 범위를 벗어났습니다. 종료 중...")
        sys.exit(1)

    success, response, message = bot.ask(final_prompt)
    if success:
        final_response = response  # Change this line
        print(f"INFO: ChatGPT의 응답: {response}")
    else:
        raise RuntimeError(message)
    
    count_yes = 0
    again_final_prompt_base = "I think you are not done yet. Please input the leftover contents."

    # Create a variable to store the concatenated responses
    concatenated_responses = ""

    while True: 
        user_input = input("\nINFO: 답변이 잘린 것 같나요? (y/n): ")
        if user_input.strip().lower() == "y":
            count_yes += 1 
            print("\nINFO: 추가 답변을 요청합니다...")
            last_response_part = response.strip().split()[-1] # Get the last word
            again_final_prompt = f"{again_final_prompt_base}" + "\n" + "However, keep in mind that you SHOULD NOT PRINT THE TEMPLATE that I gave you now on; JUST KEEP GENERATING from the truncated part. NEVER RESTART the conversation. Thank you."
            success, response, message = bot.ask(again_final_prompt)
            if success: 
                # Find the overlapping part between the last response and the new response
                overlap_start = response.find(last_response_part)
                if overlap_start != -1:
                    response = response[overlap_start + len(last_response_part):]

                concatenated_responses += response  # Append the response without the overlapping part to the concatenated_responses
                print(f"INFO: 이어진 ChatGPT의 응답: {concatenated_responses}")
            else:
                raise RuntimeError(message)
        elif user_input.strip().lower() == "n":
            break
        else: 
            print("ERROR: 잘못된 선택입니다. 다시 시도하세요.")

    # Concatenate all the response parts upto the number of times the user says yes. Direction: end to start
    final_response = final_response + "\n" + concatenated_responses

    # prompt user to choose output format
    output_format = input("\nINFO 원하시는 출력 형식 (stream / txt / md)을 선택하세요: ")

    # define maximum length of each response part to be printed at once
    max_response_length = 3000

    if output_format.lower() == "stream":
        # print response parts until the full response is generated
        i = 0
        while i < len(final_response):
            # print next response part
            response_part = final_response[i:i+max_response_length]
            print(response_part)
            i += len(response_part)

            # if there are more response parts, ask the user to continue
            if i < len(final_response):
                user_input = input("INFO: Enter 키를 눌러 계속 진행하거나 'quit'을 입력하여 종료합니다:")
                if user_input.strip().lower() == "quit":
                    break
                else: 
                    print("ERROR: 잘못된 선택입니다. 종료 중...")

    # Export the file name as same as the input file name (`file_list[user_input-1]`)
    elif output_format.lower() == "txt":
        # write response to a text file
        with open("OUTPUT.txt", "w") as f:
            f.write(final_response)
        print("INFO: OUTPUT.txt 파일이 저장되었어요.")
        files.download('./OUTPUT.txt')
    elif output_format.lower() == "md":
        # write response to a markdown file
        with open("OUTPUT.md", "w") as f:
            f.write(f"\n{final_response}\n")
        print("INFO: Output.md 파일이 저장되었어요.")
        files.download('./OUTPUT.md')
    else:
        print("ERROR: 잘못된 출력 형식을 선택하셨어요. 'stream', 'txt' 또는 'md'를 선택하세요.")


if __name__ == "__main__":
    main()

In [ ]:
#@title ## **⚠️ 예상치 못한 오류가 발생하였다면?**
#@markdown __"실행"__ 버튼을 눌러 런타임을 초기화하세요. <br/> 
#@markdown 그 후, `1번`부터 다시 진행해보세요. 

print('INFO: 코랩 런타임을 종료합니다. 1번부터 다시 진행해보세요.')
import time 
time.sleep(1)
!kill -9 -1
import os
os.kill(os.getpid(), 9)

# 레거시 버젼 테스팅
- Use ChatGPT instead of OpenAIAPI

In [ ]:
!pip install tqdm -qq
from tqdm.notebook import tqdm_notebook

packages = ['pyfiglet', 'pexpect', 'openai', 'tabulate', 'fitz', 'pdf2image', 'pytesseract', 'frontend', 'opencv-contrib-python', 'pillow', 'playwright']
for package in tqdm_notebook(packages, desc="Installing packages"):
    !pip install $package -qq &> /dev/null

!git clone https://github.com/wjgoarxiv/papersumgpt.git &> /dev/null
!chmod +x ./papersumgpt/install_old-repo.sh; ./papersumgpt/install_old-repo.sh &> /dev/null
!pip install ./papersumgpt -qq

!apt-get install poppler-utils -qq &> /dev/null # To progress PDF OCR procedures in Colab
!sudo apt-get install tesseract-ocr -qq &> /dev/null # Configure `tesseract-ocr` in Colab
import os
os.environ['PATH'] += ':/usr/local/bin'
!echo "INFO: 설치가 완료되었어요!"

In [ ]:
!playwright install &> /dev/null
!apt-get install libxtst6 libdbus-glib-1-2 &> /dev/null

In [ ]:
!apt-get update &> /dev/null
!apt-get install -y xvfb &> /dev/null
!pip install pyvirtualdisplay -qq
from pyvirtualdisplay import Display 
display = Display(visible=0, size=(1400, 900))
display.start()

!apt-get install sqlite3 &> /dev/null

import pexpect
import os
from tqdm.auto import tqdm

# Set the path to the SQLite binary
sqlite_path = "/lib/x86_64-linux-gnu/libsqlite3.so.0"

# Set the path to the SQLite database file
db_path = "////root/.local/share/chatgpt-wrapper/profiles/default/storage.db"

# Define the SQLite command to execute
sqlite_command = "{} {}"

# Start the SQLite process and connect to the database
process = pexpect.spawn("chatgpt", encoding="utf-8")

In [ ]:
import os 
import openai 

your_api = input("INFO: 복사한 API key를 붙여넣어주세요: ")
os.environ["OPENAI_API_KEY"] = your_api
os.system('cls' if os.name == 'nt' else 'clear')
print("INFO: API key가 성공적으로 등록되었어요!")

In [ ]:
from google.colab import files
uploaded = files.upload()

print("INFO: 파일(들)이 성공적으로 업로드되었어요!")

In [ ]:
#@markdown Changing the `backend` setting in the `config.yaml` file.
!cp /content/chatgpt-wrapper-5de50d92d2b9937165f22463350b1fac660e8e54/config.sample.yaml /root/.config/chatgpt-wrapper/profiles/default/config.yaml 
!sed -i 's/backend: chatgpt-browser/backend: chatgpt-api/' /root/.config/chatgpt-wrapper/profiles/default/config.yaml

In [ ]:
!apt-get install sqlite3 &> /dev/null

import pexpect
import os
from tqdm.auto import tqdm

# Set the path to the SQLite binary
sqlite_path = "/lib/x86_64-linux-gnu/libsqlite3.so.0"

# Set the path to the SQLite database file
db_path = "////root/.local/share/chatgpt-wrapper/profiles/default/storage.db"

# Define the SQLite command to execute
sqlite_command = "{} {}"

# Start the SQLite process and connect to the database
process = pexpect.spawn("chatgpt install", encoding="utf-8")
process.expect("Enter username")

# Send the username to the SQLite process and expect a prompt
process.sendline("UserID1")
process.expect("Enter email")

# Send a blank password to the SQLite process and expect a prompt
process.sendline("")
process.expect("Enter password")

# Send a blank email to the SQLite process and expect a prompt
process.sendline("")
process.expect("Provide a prompt")
print(process.before)

# Send a command to log in using pexpect
process.sendline("/login UserID1")
process.expect("userid1@")

# Send a command to exit the SQLite process
process.sendline("/exit")
process.wait()

tqdm.write("INFO: ChatGPT API를 사용할 준비가 완료되었어요.")

In [ ]:
!papersumgpt